Stage 1 training on 128*128 images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive
%cd Project/COVID-19-master
!ls

In [ ]:
# # #importing 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

Load npy files of each sizes

In [ ]:
from os import listdir
from os.path import isfile, join
xtrain_224 = np.load('data/x_train.npy')
print("RUN HOGAYA")
ytrain = np.load('data/y_train.npy')
    
xtest = np.load('data/x_test.npy')
ytest = np.load('data/y_test.npy')

RUN HOGAYA


Main code

In [ ]:
def confusion_matrix_info(y_true, y_pred, labels=['normal', 'pneumonia',  'COVID-19'],
                          title='confusion matrix'):
    import seaborn as sns
    import pandas as pd
    from sklearn.metrics import confusion_matrix, f1_score
    import matplotlib.pyplot as plt
    C2 = confusion_matrix(y_true, y_pred)
    C = pd.DataFrame(C2, columns=labels, index=labels)
    m, _ = C2.shape
    for i in range(m):
        precision = C2[i, i] / sum(C2[:, i])
        recall = C2[i, i] / sum(C2[i, :])
        f1 = 2 * precision * recall / (precision + recall)
        print('In class {}:\t total samples: {}\t true predict samples: {}\t'
              'acc={:.4f},\trecall={:.4f},\tf1-score={:.4f}'.format(
            labels[i], sum(C2[i, :]), C2[i, i], precision, recall, f1))
    print('-' * 100, '\n', 'average f1={:.4f}'.format(f1_score(y_true, y_pred, average='micro')))
 
    f, ax = plt.subplots()
    sns.heatmap(C, annot=True, ax=ax, cmap=plt.cm.binary)
    ax.set_title(title)
    ax.set_xlabel('predict')
    ax.set_ylabel('true')
    plt.savefig(title+'.jpg')

In [ ]:

resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
NUM_CLASSES = 3
DENSE_LAYER_ACTIVATION = 'softmax'



In [ ]:
#resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
NUM_CLASSES = 3
DENSE_LAYER_ACTIVATION = 'softmax'

def first_keras_model():
    model1 = Sequential()

    # 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
    # NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
    model1.add(ResNet50(include_top = False,input_shape=(128,128,3), pooling = 'avg', weights = resnet_weights_path))
    model1.add(BatchNormalization())

    # 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
    model1.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

    # Say not to train first layer (ResNet) model as it is already trained
    
    return model1



In [ ]:
def train(xtrain128, y):
    y = tf.keras.utils.to_categorical(y, 3)
    
    #TRAINING ON 128*128 WITH FREEZED RESNET LAYER
    model = first_keras_model()
    model=model
    model.layers[0].trainable = False
    print(model.summary())
    opt=tf.keras.optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    hist = model.fit(xtrain128, y, batch_size=32,  epochs=3, verbose=1)


    #FINE TUNING ON 128*128 WITH UNFREEZED RESNET LAYER
    model.layers[0].trainable = True
    print(model.summary())
    opt=tf.keras.optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    hist = model.fit(xtrain128, y, batch_size=32,  epochs=5, verbose=1)
    model.save('model1.h5')
    y_pred = model2.predict(xtrain128)
    confusion_matrix_info(np.argmax(y, axis=1), np.argmax(y_pred, axis=1),title='confusion_matrix_train')

In [ ]:
def test(xt, yt):
    model = tf.keras.models.load_model('model1.h5')
    xt = np.load('data/x_test.npy')
    yt = np.load('data/y_test.npy')
    y_pred = model.predict(xt)
    confusion_matrix_info(yt, np.argmax(y_pred, axis=1),title='confusion_matrix_test')

In [ ]:
train(xtrain_128,xtrain_224,xtrain_229, ytrain)

In [ ]:
test(xtest, ytest)